Swift AML

In [63]:
import pandas as pd
import re

# Including all SWIFT messages
swift_messages = [
    # SWIFT message 1
    """{1:F01MYMBGB2L0XXX0000000000}{2:I103HBUKGB4BXXXN}{3:{108:MT1030001}}{4:
:20:MT103 0001
:23B:CRED
:32A:210322USD5000
:50K:/DE98765432101234567890
COMMERZBANK AG
HAMBURG GERMANY
/COBADEHHXXX
:52A:/COBADEHHXXX
COMMERZBANK AG
HAMBURG GERMANY
:53A:/MYMBGB2LXXX
METRO BANK PLC
LONDON UNITED KINGDOM
:57A:/HBUKGB4BXXX
HSBC BANK PLC
LONDON UNITED KINGDOM
:59:/GB57METR12345678901234
NORDFISCH GMBH
BODENSEE STR. 226
22761 HAMBURG
GERMANY
:71A:OUR
:71F:/BIC/HBUKGB4BXXX
:71G:/INS/THIS IS A PAYMENT FOR TUNA SUPPLY
-""",
    # SWIFT message 2
    """{1:F01ABCBUS33AXXX0000000000}{2:I103HSBCHKHHHKXXXN}{3:{108:MT1030001}}{4:
:20:MT103 0001
:23B:CRED
:32A:210322USD10000
:50K:/US12345678901234567890
ABC INDUSTRIES
123 MAIN STREET
NEW YORK NY 10001
UNITED STATES
/ABCBUS33XXX
:52A:/ABCBUS33XXX
ABC BANK
NEW YORK NY
UNITED STATES
:53A:/HSBCHKHHHKXXX
HSBC HONG KONG
1 QUEEN'S ROAD CENTRAL
HONG KONG
:54A:/CITIHKHX
CITIBANK HONG KONG
3 GARDEN ROAD
CENTRAL HONG KONG
:56A:/ICBKCNBJGZU
INDUSTRIAL AND COMMERCIAL BANK OF CHINA
GUANGZHOU BRANCH
76 HUANSHI ROAD WEST
GUANGZHOU CHINA
:57A:/CITIUS33
CITIBANK NA
111 WALL STREET
NEW YORK NY 10043
UNITED STATES
:59:/CN123456789012345678
XYZ SUPPLIERS
123 HUANGPU ROAD
SHANGHAI CHINA
:71A:OUR
:71F:/BIC/HSBCHKHHHKXXX
:71G:/MSG/PAYMENT FOR GOODS
-""",
    # SWIFT message 3
    # We have changed the 50A field to reflect the SWIFT standards. 
    """{1:F01ABNANL2AXXX0000000000}{2:I103SCBLGB2LXXXXN}{3:{103:TGT}{108:MT1030001}}{4:
:20:MT103 0001
:23B:CRED
:32A:210322USD9899
:50A:/NL123456789123456789
ABN AMRO
AMSTERDAM, NETHERLANDS
/ABNANL2AXXX
:56A:/SCBLGB2LXXX
STANDARD CHARTERED BANK
LONDON, UK
:57A:/BNYMUS33XXX
BNY MELLON
NEW YORK, NY, US
:59:/NG123456789012345678
AFRICAN EXPORT-IMPORT BANK
XYZ ENTERPRISES LTD
LAGOS, NIGERIA
:70:INV NO. 12345
REF. 98765
SUPPLY OF GOODS AS PER PURCHASE ORDER NO. 54321
-""",
#Generated with AI Because the 3rd wont work.
#Swift message 4
"""{1:F01BNYNUS33AXXX0000000002}{2:I103CITIUS33XXXN}{3:{108:MT1030002}}{4:
:20:MT103 0001
:23B:CRED
:32A:210325EUR15000
:50K:/FR09876543210987654321
GHI MANUFACTURING
456 INDUSTRIAL PARK
PARIS FR 75009
FRANCE
/BNYNUS33AXXX
:52A:/BNYNUS33AXXX
BNY MELLON
NEW YORK, NY
UNITED STATES
:53A:/CITIUS33XXX
CITIBANK NEW YORK
111 WALL STREET
NEW YORK, NY
:54A:/BNPAGB22
BNP PARIBAS LONDON
10 HAREWOOD AVENUE
LONDON, UK
:56A:/BOFAUS3N
BANK OF AMERICA NA
NEW YORK BRANCH
222 BROADWAY
NEW YORK, NY, US
:57A:/CITIUS33
CITIBANK NA
111 WALL STREET
NEW YORK NY 10043
UNITED STATES
:59:/FR12345678901234567890
LMN RETAILERS
789 COMMERCIAL RD
LYON FRANCE
:71A:OUR
:71F:/BIC/CITIUS33XXX
:71G:/MSG/PAYMENT FOR SUPPLIES
-""",
#Swift message 5
"""{1:F01BNYNUS33AXXX0000000006}{2:I103CITIUS33XXXN}{3:{108:MT1030006}}{4:
:20:MT1030004
:23B:CRED
:32A:220407EUR20000
:50K:/DE89370400440532013000
CONSTRUCTIX GMBH
25 INDUSTRIAL ZONE
STUTTGART DE 70000
GERMANY
/BNYNUS33AXXX
:52A:/BNYNUS33AXXX
BNY MELLON
NEW YORK, NY
UNITED STATES
:53B:/1234567890123456//CC
FIRST BANK
123 MAIN STREET
NEW YORK, NY
:57A:/CITIUS33XXX//123456789
CITIBANK N.A.
111 WALL STREET
NEW YORK NY 10043
UNITED STATES
:59:/DE35500700100995359300
BOLT MANUFACTURING
456 BOLT STREET
FRANKFURT DE 60313
GERMANY
:70://INVOICE 1234/PROJECT 5678
PAYMENT FOR CONSTRUCTION MATERIALS
:71A:OUR
-}"""
]

Above the SWIFT messages that are used for the codes are displayed, in addition to the three provided messages two more have been generated using AI. This is done because the third message contains a lot of errors

In [64]:
import re

# This function is designed to extract specific fields from SWIFT message text.
# SWIFT messages are a standard format used by banks for financial transactions.
def extract_payment_transaction_fields(text):
    # Define a dictionary where each key is a field name and each value is a regex pattern to extract that field from the SWIFT message text.
    # These patterns are tailored to match the unique structure of SWIFT messages.
    patterns = {
        'transaction_date': r":32A:(\d{6})",  # Extracts the transaction date in YYMMDD format.
        'transaction_id': r':20:([^\s]+)',  # Extracts the transaction identifier, a unique reference.
        'transaction_message': r":71G:/INS/(.*)",  # Extracts transaction instructions, if present.
        'transaction_currency': r':32A:\d{6}([A-Z]{3})',  # Extracts the 3-letter currency code following the date.
        'transaction_amount': r':32A:\d{6}[A-Z]{3}([\d,\.]+)',  # Extracts the transaction amount, including decimals.
        'transaction_type': r':23B:([^\s]+)',  # Extracts the transaction type code.
        'originator_bic': r':50[A-K]:[\/]?([A-Z]{6}[A-Z2-9][A-NP-Z0-9]{2}([A-Z0-9]{3})?)',  # Extracts the BIC of the originator.
        'beneficiary_bic': r':59[A-Z]?:\/?([A-Z]{6}[A-Z2-9][A-NP-Z0-9]{2}([A-Z0-9]{3})?)',  # Extracts the BIC of the beneficiary.
        'originator_account_number': r':50[A-K]:[\/]?([A-Z0-9]+)',  # Extracts the originator's account number.
        'beneficiary_account_number': r':59[A-Z]?\/?([A-Z0-9]+)',  # Extracts the beneficiary's account number.
        'details_of_charges': r':71A:([^\s]+)',  # Extracts information about charges.
        'sender_to_receiver_information': r':72:([^\s\S]+)',  # Extracts additional instructions for the receiving bank.
        'intermediary_bank': r':56A:\/?([A-Z]{4}[A-Z]{2}[A-Z0-9]{2}([A-Z0-9]{3})?)',  # Extracts the intermediary bank's BIC, if any.
        'account_with_institution': r':57[A-Z]?:\/?([A-Z]{4}[A-Z]{2}[A-Z0-9]{2}([A-Z0-9]{3})?)',  # Extracts the BIC of the account with institution.
        "beneficiary_account_number": r":59:/([^\s]+)",  # Duplicate pattern, possibly an oversight.
        "beneficiary_full_name": r":59:/[^\n]+\n(.*)\n",  # Extracts the full name of the beneficiary from the structured field.
        "originator_bic": r"(?::52A:|:52D:)([^\n]+)",  # Extracts the BIC of the originator, supporting multiple field variants.
        "originator_fi_info": r':52A:.*\n((?:.+\n)+?)(?=:[0-9]{2}[A-Z]:)',  # Extracts detailed financial information about the originator.
        "beneficiary_bic": r":57A:/(.*)",  # Extracts the BIC code of the beneficiary.
        "originator_info": r":50[AK]:((?:[^\-]{1,35}\n)+)",  # Extracts multiline originator information.
        "beneficiary_info": r":59:([^\-]+)\n",  # Extracts beneficiary information, including address.
        # Additional fields aimed to cover more specific or less common information within SWIFT messages.
        'originator_branch_id': r':53[A-Z]?:\/?([A-Z]{4}[A-Z]{2}[A-Z0-9]{2}([A-Z0-9]{3})?)',
        'beneficiary_branch_id': r':54[A-Z]?:\/?([A-Z]{4}[A-Z]{2}[A-Z0-9]{2}([A-Z0-9]{3})?)',
        'third_reimbursement_institution': r':55[A-Z]?:\/?([A-Z]{4}[A-Z]{2}[A-Z0-9]{2}([A-Z0-9]{3})?)',
        'ordering_customer': r':50[A-K]:',  # Starts the pattern for the ordering customer, typically requires further context to extract.
        'beneficiary_customer': r':59[A-Z]?:',  # Starts the pattern for the beneficiary customer, also requires further context.
        'remittance_information': r':70:([\s\S]+?)\n-',  # Extracts information about the remittance, until a new field starts.
        'regulatory_reporting': r':77B:([\s\S]+?)\n-'  # Extracts regulatory reporting information, a less commonly used field.
    }

    

    extracted_fields = {}

    # Iterate over each field and its corresponding pattern in the patterns dictionary.
    for field, pattern in patterns.items():
        # Use regular expression to find all occurrences of the pattern in the text.
        # The `re.MULTILINE` flag allows `^` and `$` to match the start and end of each line.
        match = re.findall(pattern, text, re.MULTILINE)
        
        # If there is at least one match, add it to the extracted_fields dictionary.
        if match:
            extracted_fields[field] = match

    # Return the dictionary containing all the extracted fields.
    return extracted_fields

# Process each message to extract payment transaction fields using the defined function.
data = [extract_payment_transaction_fields(msg) for msg in swift_messages]

# Create a pandas DataFrame from the list of dictionaries.
# Each dictionary represents extracted fields from one SWIFT message, resulting in a table where
# each row corresponds to a message and each column to a field.
df = pd.DataFrame(data)


Above the extraction code is provided, this code extracts a lot of information from the SWIFT messages and puts them into columns of a dataframe (df) this is done using regular expressions and pattern recognition

In [65]:
#First look at the created df
df.head()

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_bic,beneficiary_bic,originator_account_number,beneficiary_account_number,...,beneficiary_full_name,originator_fi_info,originator_info,beneficiary_info,originator_branch_id,ordering_customer,beneficiary_customer,intermediary_bank,beneficiary_branch_id,remittance_information
0,[210322],[MT103],[THIS IS A PAYMENT FOR TUNA SUPPLY],[USD],[5000],[CRED],[/COBADEHHXXX],[HBUKGB4BXXX],[DE98765432101234567890],[GB57METR12345678901234],...,[NORDFISCH GMBH],[COMMERZBANK AG\nHAMBURG GERMANY\n],[/DE98765432101234567890\nCOMMERZBANK AG\nHAMB...,[/GB57METR12345678901234\nNORDFISCH GMBH\nBODE...,"[(MYMBGB2LXXX, XXX)]",[:50K:],[:59:],NaN,NaN,NaN
1,[210322],[MT103],NaN,[USD],[10000],[CRED],[/ABCBUS33XXX],[CITIUS33],[US12345678901234567890],[CN123456789012345678],...,[XYZ SUPPLIERS],[ABC BANK\nNEW YORK NY\nUNITED STATES\n],[/US12345678901234567890\nABC INDUSTRIES\n123 ...,[/CN123456789012345678\nXYZ SUPPLIERS\n123 HUA...,"[(HSBCHKHHHKX, HKX)]",[:50K:],[:59:],"[(ICBKCNBJGZU, GZU)]","[(CITIHKHX, )]",NaN
2,[210322],[MT103],NaN,[USD],[9899],[CRED],NaN,[BNYMUS33XXX],[NL123456789123456789],[NG123456789012345678],...,[AFRICAN EXPORT-IMPORT BANK],NaN,"[/NL123456789123456789\nABN AMRO\nAMSTERDAM, N...",[/NG123456789012345678],NaN,[:50A:],[:59:],"[(SCBLGB2LXXX, XXX)]",NaN,[INV NO. 12345\nREF. 98765\nSUPPLY OF GOODS AS...
3,[210325],[MT103],NaN,[EUR],[15000],[CRED],[/BNYNUS33AXXX],[CITIUS33],[FR09876543210987654321],[FR12345678901234567890],...,[LMN RETAILERS],"[BNY MELLON\nNEW YORK, NY\nUNITED STATES\n]",[/FR09876543210987654321\nGHI MANUFACTURING\n4...,[/FR12345678901234567890\nLMN RETAILERS\n789 C...,"[(CITIUS33XXX, XXX)]",[:50K:],[:59:],"[(BOFAUS3N, )]","[(BNPAGB22, )]",NaN
4,[220407],[MT1030004],NaN,[EUR],[20000],[CRED],[/BNYNUS33AXXX],[CITIUS33XXX//123456789],[DE89370400440532013000],[DE35500700100995359300],...,[BOLT MANUFACTURING],"[BNY MELLON\nNEW YORK, NY\nUNITED STATES\n]",[/DE89370400440532013000\nCONSTRUCTIX GMBH\n25...,[/DE35500700100995359300\nBOLT MANUFACTURING\n...,NaN,[:50K:],[:59:],NaN,NaN,[//INVOICE 1234/PROJECT 5678\nPAYMENT FOR CONS...


In [66]:
#Analyzing the df columns
df.columns

Index(['transaction_date', 'transaction_id', 'transaction_message',
       'transaction_currency', 'transaction_amount', 'transaction_type',
       'originator_bic', 'beneficiary_bic', 'originator_account_number',
       'beneficiary_account_number', 'details_of_charges',
       'account_with_institution', 'beneficiary_full_name',
       'originator_fi_info', 'originator_info', 'beneficiary_info',
       'originator_branch_id', 'ordering_customer', 'beneficiary_customer',
       'intermediary_bank', 'beneficiary_branch_id', 'remittance_information'],
      dtype='object')

In [67]:
#make every column a string
df = df.applymap(lambda x: x[0] if isinstance(x, list) else x)

#display the dataframe
df

C:\Users\nusre\AppData\Local\Temp\ipykernel_163408\420779606.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x[0] if isinstance(x, list) else x)


,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_bic,beneficiary_bic,originator_account_number,beneficiary_account_number,...,beneficiary_full_name,originator_fi_info,originator_info,beneficiary_info,originator_branch_id,ordering_customer,beneficiary_customer,intermediary_bank,beneficiary_branch_id,remittance_information
0,210322,MT103,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,/COBADEHHXXX,HBUKGB4BXXX,DE98765432101234567890,GB57METR12345678901234,...,NORDFISCH GMBH,COMMERZBANK AG\nHAMBURG GERMANY\n,/DE98765432101234567890\nCOMMERZBANK AG\nHAMBU...,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,"(MYMBGB2LXXX, XXX)",:50K:,:59:,NaN,NaN,NaN
1,210322,MT103,NaN,USD,10000,CRED,/ABCBUS33XXX,CITIUS33,US12345678901234567890,CN123456789012345678,...,XYZ SUPPLIERS,ABC BANK\nNEW YORK NY\nUNITED STATES\n,/US12345678901234567890\nABC INDUSTRIES\n123 M...,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,"(HSBCHKHHHKX, HKX)",:50K:,:59:,"(ICBKCNBJGZU, GZU)","(CITIHKHX, )",NaN
2,210322,MT103,NaN,USD,9899,CRED,NaN,BNYMUS33XXX,NL123456789123456789,NG123456789012345678,...,AFRICAN EXPORT-IMPORT BANK,NaN,"/NL123456789123456789\nABN AMRO\nAMSTERDAM, NE...",/NG123456789012345678,NaN,:50A:,:59:,"(SCBLGB2LXXX, XXX)",NaN,INV NO. 12345\nREF. 98765\nSUPPLY OF GOODS AS ...
3,210325,MT103,NaN,EUR,15000,CRED,/BNYNUS33AXXX,CITIUS33,FR09876543210987654321,FR12345678901234567890,...,LMN RETAILERS,"BNY MELLON\nNEW YORK, NY\nUNITED STATES\n",/FR09876543210987654321\nGHI MANUFACTURING\n45...,/FR12345678901234567890\nLMN RETAILERS\n789 CO...,"(CITIUS33XXX, XXX)",:50K:,:59:,"(BOFAUS3N, )","(BNPAGB22, )",NaN
4,220407,MT1030004,NaN,EUR,20000,CRED,/BNYNUS33AXXX,CITIUS33XXX//123456789,DE89370400440532013000,DE35500700100995359300,...,BOLT MANUFACTURING,"BNY MELLON\nNEW YORK, NY\nUNITED STATES\n",/DE89370400440532013000\nCONSTRUCTIX GMBH\n25 ...,/DE35500700100995359300\nBOLT MANUFACTURING\n4...,NaN,:50K:,:59:,NaN,NaN,//INVOICE 1234/PROJECT 5678\nPAYMENT FOR CONST...


In [68]:
#Analyzing the shape of df
df.shape

(5, 22)

In [69]:
#define a function to extract the country code from the beneficiary_account_number and originator_account_number
def extract_country_code(account_number):
    if account_number:
        return account_number[:2]
    return None

df['originator_country_code'] = df['originator_account_number'].apply(extract_country_code)
df['beneficiary_country_code'] = df['beneficiary_account_number'].apply(extract_country_code)

df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_bic,beneficiary_bic,originator_account_number,beneficiary_account_number,...,originator_info,beneficiary_info,originator_branch_id,ordering_customer,beneficiary_customer,intermediary_bank,beneficiary_branch_id,remittance_information,originator_country_code,beneficiary_country_code
0,210322,MT103,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,/COBADEHHXXX,HBUKGB4BXXX,DE98765432101234567890,GB57METR12345678901234,...,/DE98765432101234567890\nCOMMERZBANK AG\nHAMBU...,/GB57METR12345678901234\nNORDFISCH GMBH\nBODEN...,"(MYMBGB2LXXX, XXX)",:50K:,:59:,NaN,NaN,NaN,DE,GB
1,210322,MT103,NaN,USD,10000,CRED,/ABCBUS33XXX,CITIUS33,US12345678901234567890,CN123456789012345678,...,/US12345678901234567890\nABC INDUSTRIES\n123 M...,/CN123456789012345678\nXYZ SUPPLIERS\n123 HUAN...,"(HSBCHKHHHKX, HKX)",:50K:,:59:,"(ICBKCNBJGZU, GZU)","(CITIHKHX, )",NaN,US,CN
2,210322,MT103,NaN,USD,9899,CRED,NaN,BNYMUS33XXX,NL123456789123456789,NG123456789012345678,...,"/NL123456789123456789\nABN AMRO\nAMSTERDAM, NE...",/NG123456789012345678,NaN,:50A:,:59:,"(SCBLGB2LXXX, XXX)",NaN,INV NO. 12345\nREF. 98765\nSUPPLY OF GOODS AS ...,NL,NG
3,210325,MT103,NaN,EUR,15000,CRED,/BNYNUS33AXXX,CITIUS33,FR09876543210987654321,FR12345678901234567890,...,/FR09876543210987654321\nGHI MANUFACTURING\n45...,/FR12345678901234567890\nLMN RETAILERS\n789 CO...,"(CITIUS33XXX, XXX)",:50K:,:59:,"(BOFAUS3N, )","(BNPAGB22, )",NaN,FR,FR
4,220407,MT1030004,NaN,EUR,20000,CRED,/BNYNUS33AXXX,CITIUS33XXX//123456789,DE89370400440532013000,DE35500700100995359300,...,/DE89370400440532013000\nCONSTRUCTIX GMBH\n25 ...,/DE35500700100995359300\nBOLT MANUFACTURING\n4...,NaN,:50K:,:59:,NaN,NaN,//INVOICE 1234/PROJECT 5678\nPAYMENT FOR CONST...,DE,DE


Above the country codes are extracted form the originator_account_number and columns, the originator_country_code and beneficiary_country_code columns are then created contain country codes

In [70]:
import pycountry

#Define a function to get the country name from the originator_country_code and beneficiary_country_code
def get_country_name(country_code):
    try:
        return pycountry.countries.get(alpha_2=country_code).name
    except:
        return None
    
df['originator_country'] = df['originator_country_code'].apply(get_country_name)
df['beneficiary_country'] = df['beneficiary_country_code'].apply(get_country_name)

df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_bic,beneficiary_bic,originator_account_number,beneficiary_account_number,...,originator_branch_id,ordering_customer,beneficiary_customer,intermediary_bank,beneficiary_branch_id,remittance_information,originator_country_code,beneficiary_country_code,originator_country,beneficiary_country
0,210322,MT103,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,/COBADEHHXXX,HBUKGB4BXXX,DE98765432101234567890,GB57METR12345678901234,...,"(MYMBGB2LXXX, XXX)",:50K:,:59:,NaN,NaN,NaN,DE,GB,Germany,United Kingdom
1,210322,MT103,NaN,USD,10000,CRED,/ABCBUS33XXX,CITIUS33,US12345678901234567890,CN123456789012345678,...,"(HSBCHKHHHKX, HKX)",:50K:,:59:,"(ICBKCNBJGZU, GZU)","(CITIHKHX, )",NaN,US,CN,United States,China
2,210322,MT103,NaN,USD,9899,CRED,NaN,BNYMUS33XXX,NL123456789123456789,NG123456789012345678,...,NaN,:50A:,:59:,"(SCBLGB2LXXX, XXX)",NaN,INV NO. 12345\nREF. 98765\nSUPPLY OF GOODS AS ...,NL,NG,Netherlands,Nigeria
3,210325,MT103,NaN,EUR,15000,CRED,/BNYNUS33AXXX,CITIUS33,FR09876543210987654321,FR12345678901234567890,...,"(CITIUS33XXX, XXX)",:50K:,:59:,"(BOFAUS3N, )","(BNPAGB22, )",NaN,FR,FR,France,France
4,220407,MT1030004,NaN,EUR,20000,CRED,/BNYNUS33AXXX,CITIUS33XXX//123456789,DE89370400440532013000,DE35500700100995359300,...,NaN,:50K:,:59:,NaN,NaN,//INVOICE 1234/PROJECT 5678\nPAYMENT FOR CONST...,DE,DE,Germany,Germany


Using Pycountry the country codes are then matched with countries and put into the originator_country and beneficiary_country columns

In [71]:
#Having a look at the country columns
selected_columns = df[["originator_country", "beneficiary_country"]]
selected_columns


,originator_country,beneficiary_country
0,Germany,United Kingdom
1,United States,China
2,Netherlands,Nigeria
3,France,France
4,Germany,Germany


In [72]:
#Extracting the country code from the columns originator_bic and beneficiary_bic and storing them in new columns originator_country_code and beneficiary_country_code. It is the 5th and 6th character of the BIC code
df['originator_fi_country'] = df['originator_bic'].str[5:7]
df['beneficiary_fi_country'] = df['beneficiary_bic'].str[4:6]


In [73]:
import pycountry

#Creating the columns originator_fi_country and beneficiary_fi_country by defining a function to get the country name from the originator_country_code and beneficiary_country_code
def get_country_name(country_code):
    try:
        return pycountry.countries.get(alpha_2=country_code).name
    except:
        return None
    
df['originator_fi_country'] = df['originator_fi_country'].apply(get_country_name)
df['beneficiary_fi_country'] = df['beneficiary_fi_country'].apply(get_country_name)


Above similarly pycountry is used in combination with identified country codes to make a column containing country names of the financial institutions

In [74]:
import pandas as pd

# Function to extract bank name and address from the given text
def extract_bank_name_and_address(text):
    # Check if text is not NaN (float in this context)
    if pd.notna(text):
        lines = text.split("\n")
        bank_name = lines[0]
        address = " ".join(lines[1:])
        return bank_name, address
    return None, None

# Apply the fixed function to both columns
df['originator_fi_name'], df['originator_fi_address'] = zip(*df['originator_fi_info'].apply(extract_bank_name_and_address))
df['beneficiary_fi_name'], df['beneficiary_fi_address'] = zip(*df['beneficiary_info'].apply(extract_bank_name_and_address))

Above the name and address columns of the involved financial institutions are extracted and put into columns

In [75]:
#A fuction is defined to extract the name and address of the beneficiary from the beneficiary_info column
def extract_beneficiary_info(info):
    #The FI name and address are extracted 
    matches = re.findall(r"/[A-Z0-9]+\n(.*?)\n(.*?)(?=\n/|\n:|$)", info, re.DOTALL)
    if matches:
        #First match is relevant
        bank_name, address = matches[0]
        return bank_name, address
    return None, None

#Function is applied to column and stored in new columns
df['beneficiary_name'], df['beneficiary_address'] = zip(*df['beneficiary_info'].apply(extract_beneficiary_info))

#Some unnecesary space and characters are removed
df['beneficiary_name'] = df['beneficiary_name'].str.replace("\n", " ")
df['beneficiary_address'] = df['beneficiary_address'].str.replace("\n", " ")

#created columns are viewed 
print(df[['beneficiary_name', 'beneficiary_address']])



     beneficiary_name                         beneficiary_address
0      NORDFISCH GMBH     BODENSEE STR. 226 22761 HAMBURG GERMANY
1       XYZ SUPPLIERS             123 HUANGPU ROAD SHANGHAI CHINA
2                None                                        None
3       LMN RETAILERS               789 COMMERCIAL RD LYON FRANCE
4  BOLT MANUFACTURING  456 BOLT STREET FRANKFURT DE 60313 GERMANY


In [76]:
#Writing a function to get the name and address of the originator from the originator_info column
def extract_originator_info(info):
    #Searching for FI name and address
    matches = re.findall(r"/[A-Z0-9]+\n(.*?)\n(.*?)(?=\n/|\n:|$)", info, re.DOTALL)
    if matches:
        #First match is relevant
        bank_name, address = matches[0]
        return bank_name, address
    return None, None

#Function is applied to column and stored in new columns
df['originator_name'], df['originator_address'] = zip(*df['originator_info'].apply(extract_originator_info))

#Removing some unrelevant space and characters
df['originator_name'] = df['originator_name'].str.replace("\n", " ")
df['originator_address'] = df['originator_address'].str.replace("\n", " ")

#created columns are viewed
print(df[['originator_name', 'originator_address']])

     originator_name                               originator_address
0     COMMERZBANK AG                                  HAMBURG GERMANY
1     ABC INDUSTRIES  123 MAIN STREET NEW YORK NY 10001 UNITED STATES
2           ABN AMRO                           AMSTERDAM, NETHERLANDS
3  GHI MANUFACTURING        456 INDUSTRIAL PARK PARIS FR 75009 FRANCE
4   CONSTRUCTIX GMBH    25 INDUSTRIAL ZONE STUTTGART DE 70000 GERMANY


In the above two cells the information columns which contain a lot of information about both the beneficiary and the originator are used to create 4 new column that contain the address and names of the originators and beneficiaries. The example guide also mentions first, middle and last names but this information is not contained within any of the provided swift messages. Therefore, we have used the names of institutions/companies instead.

## 2) a) round amount payments

In [77]:
# Convert 'transaction_amount' to numeric, coercing errors to NaN
df['transaction_amount'] = pd.to_numeric(df['transaction_amount'], errors='coerce')

# Now apply your lambda function safely
df['round_amount'] = df['transaction_amount'].apply(lambda x: x % 1000 == 0 if pd.notnull(x) else False)

# This will add a boolean column 'round_amount' where True indicates a round amount payment, and False otherwise.
df['round_amount']

0     True
1     True
2    False
3     True
4     True
Name: round_amount, dtype: bool

## 2) b) Payments from high risk countries to tax havens

In [78]:
#load the data from "D:\school HvA\Mater Digital Driven Business\Vakken\Fintech\Tool development Sven&Nusret\Bitcoin code\High risk and Tax Havens.csv" use ; to separate the columns
df2 = pd.read_csv(r"C:\Users\nusre\Downloads\High risk and Tax Havens.csv", sep=';')
df2

,high_risk_countries,tax_havens
0,Democratic People's Republic of Korea,Vietnam
1,Iran,Angola
2,Myanmar,Bolivia
3,Bulgaria,United Arab Emirates
4,Burkina Faso,Algeria
5,Cameroon,Puerto Rico
6,Croatia,St. Kitts and Nevis
7,Democratic Republic of Congo,Antigua and Barbuda
8,Haiti,Curacao
9,Jamaica,Vanuatu


In [79]:
# Extract high-risk countries and tax havens from df2
high_risk_countries = df2['high_risk_countries'].tolist()
tax_havens = df2['tax_havens'].tolist()

# Function to identify payments from high-risk countries to tax havens
def is_highrisk_to_taxhaven(row):
    originator_is_high_risk = row["originator_country"] in high_risk_countries
    beneficiary_is_tax_haven = row["beneficiary_country"] in tax_havens
    return originator_is_high_risk and beneficiary_is_tax_haven

# Apply the function to df
df['highrisk_to_taxhaven'] = df.apply(is_highrisk_to_taxhaven, axis=1)

# Display the DataFrame to verify the new column
df['highrisk_to_taxhaven']


0    False
1    False
2    False
3    False
4    False
Name: highrisk_to_taxhaven, dtype: bool

In [80]:
def is_highrisk_to_taxhaven(row):
    return row["beneficiary_country"] in high_risk_countries or row["originator_country"] and row["originator_country"] in tax_havens or row["beneficiary_country"]

In [81]:
# Making a function that identiefies smurfing
def identify_and_mark_smurfing(df, amount_threshold=1000, time_frame='1D', transaction_count_threshold=5):
    """
    Enhances the identify_smurfing function to not only identify potential smurfing transactions but
    also mark each transaction in the original dataframe with a new 'smurfing' column indicating whether
    the transaction is potentially part of smurfing activity.
    """
    # Ensure transaction_date is a datetime
    df['transaction_date'] = pd.to_datetime(df['transaction_date'])
    
    # Group by account and date, then aggregate
    aggregated = df.groupby([pd.Grouper(key='transaction_date', freq=time_frame), 'originator_account_number']).agg(
        total_amount=pd.NamedAgg(column='transaction_amount', aggfunc='sum'),
        transaction_count=pd.NamedAgg(column='transaction_amount', aggfunc='count'),
        max_transaction=pd.NamedAgg(column='transaction_amount', aggfunc='max')
    ).reset_index()
    
    # Mark groups that meet smurfing criteria
    aggregated['is_smurfing'] = (aggregated['total_amount'] > amount_threshold) & (aggregated['transaction_count'] > transaction_count_threshold) & (aggregated['max_transaction'] <= amount_threshold)
    
    # Map aggregated smurfing information back to the original dataframe
    df['smurfing'] = False # Default value
    for _, row in aggregated.iterrows():
        if row['is_smurfing']:
            condition = (df['transaction_date'].dt.floor(time_frame) == row['transaction_date']) & (df['originator_account_number'] == row['originator_account_number'])
            df.loc[condition, 'smurfing'] = True

     
    return df

df = identify_and_mark_smurfing(df)

C:\Users\nusre\AppData\Local\Temp\ipykernel_163408\2555621216.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['transaction_date'] = pd.to_datetime(df['transaction_date'])


In [82]:
df['smurfing']

0    False
1    False
2    False
3    False
4    False
Name: smurfing, dtype: bool

The `identify_and_mark_smurfing` function flags transactions in a DataFrame that are potentially part of smurfing activities. Smurfing is a method of money laundering where large sums are divided into smaller, less suspicious amounts.

## Parameters

- `df`: DataFrame with transaction data. Requires `transaction_date`, `originator_account_number`, and `transaction_amount` columns.
- `amount_threshold` (default: `1000`): Minimum sum of transactions within a timeframe to consider as potential smurfing.
- `time_frame` (default: `'1D'`): Timeframe for aggregating transactions (e.g., '1D' for one day).
- `transaction_count_threshold` (default: `5`): Minimum number of transactions within a timeframe to flag as potential smurfing.

## Workflow

1. **Datetime Conversion**: Converts `transaction_date` to datetime format.
2. **Aggregation**: Groups transactions by account and date, calculating total amount, transaction count, and maximum transaction value.
3. **Smurfing Detection**: Adds `is_smurfing` to flag groups with total amounts and transaction counts exceeding thresholds, but with individual transactions not exceeding the amount threshold.
4. **Mark Transactions**: Updates the original DataFrame with a `smurfing` boolean column to indicate potential smurfing activities.

In [83]:
# Convert the 'transaction_date' column to datetime format
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# Display the DataFrame to verify the change
df


,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_bic,beneficiary_bic,originator_account_number,beneficiary_account_number,...,originator_fi_address,beneficiary_fi_name,beneficiary_fi_address,beneficiary_name,beneficiary_address,originator_name,originator_address,round_amount,highrisk_to_taxhaven,smurfing
0,2022-03-21,MT103,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,/COBADEHHXXX,HBUKGB4BXXX,DE98765432101234567890,GB57METR12345678901234,...,HAMBURG GERMANY,/GB57METR12345678901234,NORDFISCH GMBH BODENSEE STR. 226 22761 HAMBURG...,NORDFISCH GMBH,BODENSEE STR. 226 22761 HAMBURG GERMANY,COMMERZBANK AG,HAMBURG GERMANY,True,False,False
1,2022-03-21,MT103,NaN,USD,10000,CRED,/ABCBUS33XXX,CITIUS33,US12345678901234567890,CN123456789012345678,...,NEW YORK NY UNITED STATES,/CN123456789012345678,XYZ SUPPLIERS 123 HUANGPU ROAD SHANGHAI CHINA ...,XYZ SUPPLIERS,123 HUANGPU ROAD SHANGHAI CHINA,ABC INDUSTRIES,123 MAIN STREET NEW YORK NY 10001 UNITED STATES,True,False,False
2,2022-03-21,MT103,NaN,USD,9899,CRED,NaN,BNYMUS33XXX,NL123456789123456789,NG123456789012345678,...,None,/NG123456789012345678,,None,None,ABN AMRO,"AMSTERDAM, NETHERLANDS",False,False,False
3,2025-03-21,MT103,NaN,EUR,15000,CRED,/BNYNUS33AXXX,CITIUS33,FR09876543210987654321,FR12345678901234567890,...,"NEW YORK, NY UNITED STATES",/FR12345678901234567890,LMN RETAILERS 789 COMMERCIAL RD LYON FRANCE :7...,LMN RETAILERS,789 COMMERCIAL RD LYON FRANCE,GHI MANUFACTURING,456 INDUSTRIAL PARK PARIS FR 75009 FRANCE,True,False,False
4,2007-04-22,MT1030004,NaN,EUR,20000,CRED,/BNYNUS33AXXX,CITIUS33XXX//123456789,DE89370400440532013000,DE35500700100995359300,...,"NEW YORK, NY UNITED STATES",/DE35500700100995359300,BOLT MANUFACTURING 456 BOLT STREET FRANKFURT D...,BOLT MANUFACTURING,456 BOLT STREET FRANKFURT DE 60313 GERMANY,CONSTRUCTIX GMBH,25 INDUSTRIAL ZONE STUTTGART DE 70000 GERMANY,True,False,False


## 2) d) Nesting


### 1) First approach

In [84]:
# Function to detect nesting
def detect_nesting(swift_messages):
    # Pattern to find the fields indicating intermediary banks
    pattern = r":(?:52|53|54|55|56|57)A:"
    matches = re.findall(pattern, swift_messages)
    # Nesting is present if more than one intermediary bank field is found
    return len(matches) > 3

# Apply the function to each SWIFT message
nesting_results = [detect_nesting(message) for message in swift_messages]

#Add the results to the dataframe
df['nesting'] = nesting_results
nesting_results

[False, True, False, True, False]

The `detect_nesting` function is designed to identify "nesting" within SWIFT messages. Nesting occurs when transactions are routed through multiple intermediary banks, often to obscure the transaction path. This can be an indicator of money laundering or attempts to circumvent regulatory scrutiny.

## Input

- `swift_messages`: A string or a collection of strings representing SWIFT messages.

## Functionality

1. **Pattern Matching**: Searches for fields that indicate intermediary banks (fields 52A to 57A) within the SWIFT message using regular expressions.
2. **Nesting Detection**: Considers nesting to be present if more than three intermediary bank fields are found, suggesting the transaction path involves multiple banks.


### 2) Second approach

In [85]:
import re

def calculate_nesting_score(block):
    # Score systeem op basis van complexiteit van patronen
    score = 0
    
    # Patronen en hun bijbehorende punten
    patterns = [
        (re.compile(r':\d{2}[A-Z]:(?:.*\n){2,}:\d{2}[A-Z]:'), 5),  # Zeer complexe structuren
        (re.compile(r':\d{2}[A-Z]:.*?//.*'), 3),  # Meerdere lijnen/sub-elementen
        (re.compile(r':\d{2}[A-Z]:.*?/\d+.*'), 2),  # Accountnummers/identificatoren
        (re.compile(r':\d{2}[A-Z]:[^\n]+?\n[^\n]+'), 1),  # Velden over meerdere regels
        (re.compile(r':\d{2}[A-Z]:(?:[^/]+?/){2,}'), 2),  # Meerdere '/' scheidingstekens
    ]
    
    for pattern, points in patterns:
        matches = pattern.findall(block)
        score += len(matches) * points
    
    return score

def detect_advanced_nesting_with_scores(swift_messages):
    results = []
    for message in swift_messages:
        # Extract Blok 4
        block_4_start = message.find("{4:\n") + 4
        block_4_end = message.find("-}", block_4_start)
        block_4_content = message[block_4_start:block_4_end]
        
        # Bereken de score voor Blok 4
        score = calculate_nesting_score(block_4_content)
        
        # Controleer of de score de drempelwaarde overschrijdt
        results.append(score >= 16)
    
    return results

# Voer de detectiefunctie uit voor de gegeven SWIFT-berichten
nesting_detection_results = detect_advanced_nesting_with_scores(swift_messages)
for index, result in enumerate(nesting_detection_results):
    print(f"Message {index + 1} has nesting: {result}")

#Add the result to the dataframe 
df['advanced_nesting'] = nesting_detection_results


Message 1 has nesting: False
Message 2 has nesting: False
Message 3 has nesting: False
Message 4 has nesting: False
Message 5 has nesting: True


The above Python code introduces a method to calculate and detect advanced nesting within SWIFT messages, utilizing a scoring system based on pattern complexity.

## Components

- **`calculate_nesting_score(block)`:** Calculates a nesting score for a given block of text from a SWIFT message.
- **`detect_advanced_nesting_with_scores(swift_messages)`:** Evaluates SWIFT messages to determine if they exhibit advanced nesting, based on predefined score thresholds.

## Scoring System

The nesting score is determined by identifying specific patterns within the SWIFT message block, with each pattern assigned a different point value:
- Very complex structures: 5 points
- Multiple lines/sub-elements: 3 points
- Account numbers/identifiers: 2 points
- Fields spanning multiple lines: 1 point
- Multiple '/' separators: 2 points

## Detection Process

1. **Extract Block 4:** Isolates Block 4 from a SWIFT message, the primary content area for transaction details.
2. **Calculate Score:** Utilizes `calculate_nesting_score` to assign a score to the block based on the complexity and quantity of identified patterns.
3. **Threshold Check:** Determines if the calculated score meets or exceeds a threshold (16 points) indicative of advanced nesting.


## 2) e) Non-adherence to FATF Recommendation 16

In [86]:
# Function to check compliance with FATF Recommendation 16
def check_compliance(row):
    #List of required fields
    required_fields = ['beneficiary_name', 'beneficiary_address', 'originator_name', 'originator_address',
                       'originator_fi_country', 'beneficiary_fi_country', 'originator_fi_name', 
                       'originator_fi_address', 'beneficiary_fi_name', 'beneficiary_fi_address']
    
    # Check that all required fields are present and not empty
    for field in required_fields:
        if pd.isna(row[field]) or row[field] == '':
            # If a required field is missing or empty, return False
            return False
    
    # If all required fields are present and not empty, return True
    return True

# Adding a compliance score column to the data frame
df['FATF-16_compliance'] = df.apply(check_compliance, axis=1)

# View the resulting data frame with the compliance score
print(df[['beneficiary_name', 'beneficiary_address', 'originator_name', 'originator_address',
          'originator_fi_country', 'beneficiary_fi_country', 'originator_fi_name', 
          'originator_fi_address', 'beneficiary_fi_name', 'beneficiary_fi_address', 'FATF-16_compliance']])

df


     beneficiary_name                         beneficiary_address  \
0      NORDFISCH GMBH     BODENSEE STR. 226 22761 HAMBURG GERMANY   
1       XYZ SUPPLIERS             123 HUANGPU ROAD SHANGHAI CHINA   
2                None                                        None   
3       LMN RETAILERS               789 COMMERCIAL RD LYON FRANCE   
4  BOLT MANUFACTURING  456 BOLT STREET FRANKFURT DE 60313 GERMANY   

     originator_name                               originator_address  \
0     COMMERZBANK AG                                  HAMBURG GERMANY   
1     ABC INDUSTRIES  123 MAIN STREET NEW YORK NY 10001 UNITED STATES   
2           ABN AMRO                           AMSTERDAM, NETHERLANDS   
3  GHI MANUFACTURING        456 INDUSTRIAL PARK PARIS FR 75009 FRANCE   
4   CONSTRUCTIX GMBH    25 INDUSTRIAL ZONE STUTTGART DE 70000 GERMANY   

  originator_fi_country beneficiary_fi_country originator_fi_name  \
0               Germany         United Kingdom     COMMERZBANK AG   
1       

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_bic,beneficiary_bic,originator_account_number,beneficiary_account_number,...,beneficiary_name,beneficiary_address,originator_name,originator_address,round_amount,highrisk_to_taxhaven,smurfing,nesting,advanced_nesting,FATF-16_compliance
0,2022-03-21,MT103,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,/COBADEHHXXX,HBUKGB4BXXX,DE98765432101234567890,GB57METR12345678901234,...,NORDFISCH GMBH,BODENSEE STR. 226 22761 HAMBURG GERMANY,COMMERZBANK AG,HAMBURG GERMANY,True,False,False,False,False,True
1,2022-03-21,MT103,NaN,USD,10000,CRED,/ABCBUS33XXX,CITIUS33,US12345678901234567890,CN123456789012345678,...,XYZ SUPPLIERS,123 HUANGPU ROAD SHANGHAI CHINA,ABC INDUSTRIES,123 MAIN STREET NEW YORK NY 10001 UNITED STATES,True,False,False,True,False,True
2,2022-03-21,MT103,NaN,USD,9899,CRED,NaN,BNYMUS33XXX,NL123456789123456789,NG123456789012345678,...,None,None,ABN AMRO,"AMSTERDAM, NETHERLANDS",False,False,False,False,False,False
3,2025-03-21,MT103,NaN,EUR,15000,CRED,/BNYNUS33AXXX,CITIUS33,FR09876543210987654321,FR12345678901234567890,...,LMN RETAILERS,789 COMMERCIAL RD LYON FRANCE,GHI MANUFACTURING,456 INDUSTRIAL PARK PARIS FR 75009 FRANCE,True,False,False,True,False,True
4,2007-04-22,MT1030004,NaN,EUR,20000,CRED,/BNYNUS33AXXX,CITIUS33XXX//123456789,DE89370400440532013000,DE35500700100995359300,...,BOLT MANUFACTURING,456 BOLT STREET FRANKFURT DE 60313 GERMANY,CONSTRUCTIX GMBH,25 INDUSTRIAL ZONE STUTTGART DE 70000 GERMANY,True,False,False,False,True,True


In [87]:
# Calculate compliancescore of FATF recommendation 16
def calculate_compliance_score(row):
    compliance_score = 100 
    # Controlling if all required fiels are present
    required_fields = ['beneficiary_name', 'beneficiary_address', 'originator_name', 'originator_address',
                       'originator_fi_country', 'beneficiary_fi_country', 'originator_fi_name', 
                       'originator_fi_address', 'beneficiary_fi_name', 'beneficiary_fi_address']
    
    for field in required_fields:
        if pd.isna(row[field]) or row[field] == '':
            # if a required field is not present lessen score by 10
            compliance_score -= 10
    
    return max(compliance_score, 0)  # Makes sure score does not become negative

# Adding compliance_score column to the df
df['compliance_score'] = df.apply(calculate_compliance_score, axis=1)

# Printing df 
print(df[['beneficiary_name', 'beneficiary_address', 'originator_name', 'originator_address',
          'originator_fi_country', 'beneficiary_fi_country', 'originator_fi_name', 
          'originator_fi_address', 'beneficiary_fi_name', 'beneficiary_fi_address', 'compliance_score']])
df


     beneficiary_name                         beneficiary_address  \
0      NORDFISCH GMBH     BODENSEE STR. 226 22761 HAMBURG GERMANY   
1       XYZ SUPPLIERS             123 HUANGPU ROAD SHANGHAI CHINA   
2                None                                        None   
3       LMN RETAILERS               789 COMMERCIAL RD LYON FRANCE   
4  BOLT MANUFACTURING  456 BOLT STREET FRANKFURT DE 60313 GERMANY   

     originator_name                               originator_address  \
0     COMMERZBANK AG                                  HAMBURG GERMANY   
1     ABC INDUSTRIES  123 MAIN STREET NEW YORK NY 10001 UNITED STATES   
2           ABN AMRO                           AMSTERDAM, NETHERLANDS   
3  GHI MANUFACTURING        456 INDUSTRIAL PARK PARIS FR 75009 FRANCE   
4   CONSTRUCTIX GMBH    25 INDUSTRIAL ZONE STUTTGART DE 70000 GERMANY   

  originator_fi_country beneficiary_fi_country originator_fi_name  \
0               Germany         United Kingdom     COMMERZBANK AG   
1       

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_bic,beneficiary_bic,originator_account_number,beneficiary_account_number,...,beneficiary_address,originator_name,originator_address,round_amount,highrisk_to_taxhaven,smurfing,nesting,advanced_nesting,FATF-16_compliance,compliance_score
0,2022-03-21,MT103,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,/COBADEHHXXX,HBUKGB4BXXX,DE98765432101234567890,GB57METR12345678901234,...,BODENSEE STR. 226 22761 HAMBURG GERMANY,COMMERZBANK AG,HAMBURG GERMANY,True,False,False,False,False,True,100
1,2022-03-21,MT103,NaN,USD,10000,CRED,/ABCBUS33XXX,CITIUS33,US12345678901234567890,CN123456789012345678,...,123 HUANGPU ROAD SHANGHAI CHINA,ABC INDUSTRIES,123 MAIN STREET NEW YORK NY 10001 UNITED STATES,True,False,False,True,False,True,100
2,2022-03-21,MT103,NaN,USD,9899,CRED,NaN,BNYMUS33XXX,NL123456789123456789,NG123456789012345678,...,None,ABN AMRO,"AMSTERDAM, NETHERLANDS",False,False,False,False,False,False,40
3,2025-03-21,MT103,NaN,EUR,15000,CRED,/BNYNUS33AXXX,CITIUS33,FR09876543210987654321,FR12345678901234567890,...,789 COMMERCIAL RD LYON FRANCE,GHI MANUFACTURING,456 INDUSTRIAL PARK PARIS FR 75009 FRANCE,True,False,False,True,False,True,100
4,2007-04-22,MT1030004,NaN,EUR,20000,CRED,/BNYNUS33AXXX,CITIUS33XXX//123456789,DE89370400440532013000,DE35500700100995359300,...,456 BOLT STREET FRANKFURT DE 60313 GERMANY,CONSTRUCTIX GMBH,25 INDUSTRIAL ZONE STUTTGART DE 70000 GERMANY,True,False,False,False,True,True,100


The function `check_compliance` is designed to assess compliance with the Financial Action Task Force (FATF) Recommendation 16, also known as the "Travel Rule," within transaction data. This recommendation requires specific information to accompany transfers to help prevent money laundering and terrorist financing.

## Key Features

- **`check_compliance(row)`:** Evaluates if a transaction record meets the compliance requirements by checking for the presence and completeness of necessary fields.

## Compliance Criteria

The function verifies the presence and non-emptiness of the following fields, which are essential for adhering to the FATF Recommendation 16:
- Beneficiary's name and address
- Originator's name and address
- Information about the financial institutions (FI) involved, including the country, name, and address for both the originator's and beneficiary's FIs

## Process

1. **Field Verification:** Iterates through the list of required fields, checking each for presence (non-NA) and completeness (not empty).
2. **Compliance Decision:** Returns `False` if any required field is missing or empty, indicating non-compliance. Otherwise, it returns `True`, signaling full compliance.


## 2) f) Shell company characteristics using address and name data

In [88]:
# List of simplistic generic company indicators and tax haven indicators (very basic and for demonstration only)
#load the data from "D:\school HvA\Mater Digital Driven Business\Vakken\Fintech\Tool development Sven&Nusret\Bitcoin code\generic_name_indicators.csv" use ; to separate the columns
generic_name_indicators = pd.read_csv(r"C:\Users\nusre\Downloads\generic_name_indicators.csv", sep=';')

# Function to check for potential shell companies
def check_for_shell_company(swift_message):
    # Convert to lower case for case-insensitive matching
    message_lower = swift_message.lower()

    # Check for generic names
    for indicator in generic_name_indicators:
        if str(indicator) in message_lower:
            return True

    # Check for addresses in tax havens
    for country in tax_havens:
        if str(country) in message_lower:
            return True

    return False

# Apply the function to each SWIFT message
shell_company_results = [check_for_shell_company(message) for message in swift_messages]

shell_company_results

#Add the result to the dataframe
df['shell_company'] = shell_company_results

df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_bic,beneficiary_bic,originator_account_number,beneficiary_account_number,...,originator_name,originator_address,round_amount,highrisk_to_taxhaven,smurfing,nesting,advanced_nesting,FATF-16_compliance,compliance_score,shell_company
0,2022-03-21,MT103,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000,CRED,/COBADEHHXXX,HBUKGB4BXXX,DE98765432101234567890,GB57METR12345678901234,...,COMMERZBANK AG,HAMBURG GERMANY,True,False,False,False,False,True,100,False
1,2022-03-21,MT103,NaN,USD,10000,CRED,/ABCBUS33XXX,CITIUS33,US12345678901234567890,CN123456789012345678,...,ABC INDUSTRIES,123 MAIN STREET NEW YORK NY 10001 UNITED STATES,True,False,False,True,False,True,100,False
2,2022-03-21,MT103,NaN,USD,9899,CRED,NaN,BNYMUS33XXX,NL123456789123456789,NG123456789012345678,...,ABN AMRO,"AMSTERDAM, NETHERLANDS",False,False,False,False,False,False,40,True
3,2025-03-21,MT103,NaN,EUR,15000,CRED,/BNYNUS33AXXX,CITIUS33,FR09876543210987654321,FR12345678901234567890,...,GHI MANUFACTURING,456 INDUSTRIAL PARK PARIS FR 75009 FRANCE,True,False,False,True,False,True,100,False
4,2007-04-22,MT1030004,NaN,EUR,20000,CRED,/BNYNUS33AXXX,CITIUS33XXX//123456789,DE89370400440532013000,DE35500700100995359300,...,CONSTRUCTIX GMBH,25 INDUSTRIAL ZONE STUTTGART DE 70000 GERMANY,True,False,False,False,True,True,100,False


## Functionality
check_for_shell_company(swift_message)
## Purpose: 
Checks if a SWIFT message contains potential indicators of a shell company by looking for predefined generic names or addresses in tax havens.
## Process:
Case Insensitive Matching: Converts the SWIFT message to lower case to ensure matching is case-insensitive.
Check Generic Names: Iterates over the list of generic name indicators, flagging a message if any indicator is found.
Check Tax Haven Addresses: Similarly checks for any mention of countries known as tax havens.
Returns: True if any indicator is found, otherwise False.

## 2) g) Trade based money laundering

In [89]:
def detect_tbml_risk_indicators(swift_message):
    risk_indicators = []

    # Unusual pricing and over/under-invoicing
    amount_matches = re.findall(r':32A:(\d{6}[A-Z]{3})\d+', swift_message)  # Exclude the currency code from the amount matches
    if amount_matches:
        amounts = [float(amount[:-3]) for amount in amount_matches]  # Remove the last 3 characters (currency code)
        if max(amounts) - min(amounts) > 10000:  # Adjust threshold as needed
            risk_indicators.append("Unusual pricing or over/under-invoicing detected")

    # Multiple invoicing
    if swift_message.count(':20:') > 1:
        risk_indicators.append("Multiple invoicing detected")

    # Use of shell companies or fronts
    if re.search(r'/[A-Z]{4}[A-Z0-9]{2}\w{11}(XXX)?\n', swift_message):
        risk_indicators.append("Use of shell companies or fronts detected")

    # Complex supply chains
    if swift_message.count(':57A:') > 1:
        risk_indicators.append("Complex supply chains detected")

    # Third-party payments
    if re.search(r':56A:/.+\n', swift_message):
        risk_indicators.append("Third-party payments detected")

    return risk_indicators

for idx, swift_message in enumerate(swift_messages):
    indicators = detect_tbml_risk_indicators(swift_message)
    print(f"Risk indicators for SWIFT message {idx+1}:")
    if indicators:
        for indicator in indicators:
            print(f"- {indicator}")
    else:
        print("No risk indicators detected.")
    print()

Risk indicators for SWIFT message 1:
No risk indicators detected.

Risk indicators for SWIFT message 2:
- Third-party payments detected

Risk indicators for SWIFT message 3:
- Third-party payments detected

Risk indicators for SWIFT message 4:
- Third-party payments detected

Risk indicators for SWIFT message 5:
No risk indicators detected.



This function, detect_tbml_risk_indicators, is designed to analyze SWIFT messages for potential risk indicators of trade-based money laundering (TBML). It works by checking for certain patterns within the text of each message that may signify suspicious activity. Here's a breakdown of how it operates:

## Function Definition
Parameter: The function takes a single parameter, swift_message, which is expected to be a string containing the content of a SWIFT message.
Process Initialization: It starts by creating an empty list, risk_indicators, to store any detected TBML risk indicators.

Unusual Pricing and Over/Under-Invoicing:

It looks for patterns matching the format :32A: followed by a 6-digit date, a 3-letter currency code, and an amount. It uses regular expressions (re.findall) to find all matches but excludes the currency code from these matches.
It converts the extracted amounts (excluding currency codes) into floats and checks if the difference between the maximum and minimum amounts exceeds a certain threshold (here, 10000). If so, it adds a warning about unusual pricing or potential over/under-invoicing.
Multiple Invoicing:

Checks if the message contains more than one instance of the :20: tag, which would indicate multiple invoices are being referenced in a single SWIFT message. This could be a red flag for TBML.

Use of Shell Companies or Fronts:

Searches for a pattern that could indicate the use of a shell company or a front. The pattern looks for a specific format that might represent a bank or company identifier (potentially a shell company) within the message.
Complex Supply Chains:

Identifies complex supply chains by counting the occurrences of the :57A: tag. More than one occurrence could suggest a complex supply chain, which is a risk indicator for TBML.
Third-Party Payments:

Detects third-party payments by searching for the :56A: tag followed by certain patterns. This can be indicative of payments not directly related to the parties involved in the trade, which is another TBML risk indicator.

In [90]:
# Function to detect TBML risk indicators
def detect_tbml_risk_indicators(swift_message):
    risk_indicators = []

    # Unusual pricing and over/under-invoicing
    amount_matches = re.findall(r':32A:(\d{6}[A-Z]{3})\d+', swift_message)  # Exclude the currency code from the amount matches
    if amount_matches:
        amounts = [float(amount[:-3]) for amount in amount_matches]  # Remove the last 3 characters (currency code)
        if max(amounts) - min(amounts) > 10000:  # Adjust threshold as needed
            risk_indicators.append("Unusual pricing or over/under-invoicing detected")

    # Multiple invoicing
    if swift_message.count(':20:') > 1:
        risk_indicators.append("Multiple invoicing detected")

    # Use of shell companies or fronts
    if re.search(r'/[A-Z]{4}[A-Z0-9]{2}\w{11}(XXX)?\n', swift_message):
        risk_indicators.append("Use of shell companies or fronts detected")

    # Complex supply chains
    if swift_message.count(':57A:') > 1:
        risk_indicators.append("Complex supply chains detected")

    # Third-party payments
    if re.search(r':56A:/.+\n', swift_message):
        risk_indicators.append("Third-party payments detected")

    return {indicator: indicator in risk_indicators for indicator in [
        "Unusual pricing or over/under-invoicing detected",
        "Multiple invoicing detected",
        "Use of shell companies or fronts detected",
        "Complex supply chains detected",
        "Third-party payments detected"
    ]}

# Create a dictionary to store the results of risk indicators for each SWIFT message
results = {indicator: [] for indicator in [
    "Unusual pricing or over/under-invoicing detected",
    "Multiple invoicing detected",
    "Use of shell companies or fronts detected",
    "Complex supply chains detected",
    "Third-party payments detected"
]}

# Iterate over each SWIFT message and detect risk indicators
for swift_message in swift_messages:
    indicators = detect_tbml_risk_indicators(swift_message)
    for indicator, value in indicators.items():
        results[indicator].append(value)

# Create DataFrame from results dictionary
risk_indicators = pd.DataFrame(results)

#Add risk indicators to the dataframe
df = pd.concat([df, risk_indicators], axis=1)


# Conclusion

In [91]:
# Making the detect_money_laundering function which collect the suspicious activities identified so far and determines a threshold to flag a transaction as money laundering or not

def detect_money_laundering(row, threshold=3):
    """
    Detect potential money laundering based on various indicators.
    Each row is evaluated, and if the number of positive indicators
    exceeds the threshold, it flags the transaction as suspicious.

    Parameters:
    - row: The row of the dataframe being evaluated.
    - threshold: The minimum number of positive indicators to flag
                 the transaction as suspicious. Default is 3.

    Returns:
    - True if the transaction is potentially suspicious,
      False otherwise.
    """
    indicators = [
        'round_amount',
        'highrisk_to_taxhaven',
        'smurfing',
        'nesting',
        'shell_company',
        # Assuming you've converted the text-based TBML indicators to boolean columns
        'Unusual pricing or over/under-invoicing detected',
        'Use of shell companies or fronts detected',
        'Complex supply chains detected',
        'Third-party payments detected'
    ]
    # Count how many indicators are True for the current row
    count_true_indicators = sum(row[indicator] for indicator in indicators)


    # Check against the FATF-16 compliance; if compliant, might reduce suspicion
    fatf_16_compliance = 'FATF-16_compliance'
    if row[fatf_16_compliance]:
        count_true_indicators -= 1  # Adjust the count based on compliance
        
    #if FATF-16_compliance is false add 1 to the count
    else:
        count_true_indicators += 1 

    # Return True if the count exceeds the threshold, indicating suspicion
    return count_true_indicators >= threshold

# Apply the detection function to each row and create a new column with the results
df['potential_money_laundering'] = df.apply(detect_money_laundering, axis=1, threshold=3)

# This will add a 'potential_money_laundering' column indicating True for rows
# where the number of positive indicators meets or exceeds the specified threshold.


This Python function, detect_money_laundering, is designed to analyze financial transactions for potential signs of money laundering. It operates on a row-by-row basis within a dataframe, assessing each transaction against a set of predefined indicators. Here's a detailed explanation:

## Function Definition
Parameters:
row: A single row of a pandas dataframe, representing a financial transaction. This row contains data that the function evaluates against various indicators of money laundering.

threshold: The minimum number of positive indicators required to flag a transaction as suspicious. It defaults to 3 if not specified.

## Description
The function works by evaluating each transaction for multiple potential indicators of money laundering. These indicators are:

round_amount: Transactions involving round amounts could be indicative of structuring or attempting to avoid reporting thresholds.

highrisk_to_taxhaven: Transactions directed to or coming from high-risk jurisdictions or tax havens might indicate an attempt to obscure the origin or destination of illicit funds.

smurfing: A technique that involves breaking down large transactions into smaller ones to evade detection.

nesting: The use of intermediary financial institutions to layer transactions, complicating the traceability of funds.

shell_company: Utilizing companies with no active business operations or assets as a means to hide ownership or the true nature of transactions.

Text-based TBML Indicators: These are converted into boolean (True/False) columns in the dataset, indicating the presence or absence of trade-based money laundering risk factors such as unusual pricing, complex supply chains, and third-party payments.

The function counts the number of these indicators that are true for a given transaction.

In [92]:
df['potential_money_laundering']

0    False
1    False
2     True
3    False
4    False
Name: potential_money_laundering, dtype: bool

## Based on the threshold of at least 3 suspicious activities linked to a transaction only the third transaction is flagged as suspicious, this i mainly caused by a lot of errorr/faulty information in that SWIFT message